In [ ]:
# This is necessary to recognize the modules
import os
import sys
import datetime
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
from core.backtesting import BacktestingEngine

backtesting = BacktestingEngine(root_path=root_path, load_cached_data=True)

In [3]:
backtesting._dt_bt.backtesting_data_provider.get_candles_df("binance_perpetual", "1000PEPE-USDT", "1m")

,timestamp,open,high,low,close,volume,quote_asset_volume,n_trades,taker_buy_base_volume,taker_buy_quote_volume
2024-08-24 03:15:00,1724469300,0.0089314,0.0089327,0.0089179,0.0089306,61926117,552595.8614473,1005,47331769,422323.0680508
2024-08-24 03:16:00,1724469360,0.0089307,0.0089445,0.0089253,0.0089438,54635200,488157.4309937,980,38233612,341665.2255694
2024-08-24 03:17:00,1724469420,0.0089444,0.0089507,0.0089423,0.0089454,32141621,287538.4226658,728,17000357,152088.2780939
2024-08-24 03:18:00,1724469480,0.0089455,0.0089476,0.0089356,0.0089364,34395852,307560.8648765,530,7920616,70840.185985
2024-08-24 03:19:00,1724469540,0.0089364,0.0089375,0.0089221,0.0089227,32721732,292230.4915685,735,14914945,133158.3491865
...,...,...,...,...,...,...,...,...,...,...
2024-08-31 03:07:00,1725073620,0.0077644,0.0077644,0.0077601,0.0077619,9939964,77163.0860634,289,3601609,27958.4468139
2024-08-31 03:08:00,1725073680,0.0077623,0.0077623,0.0077538,0.0077564,19693057,152775.1236712,574,5719436,44370.0776795
2024-08-31 03:09:00,1725073740,0.0077564,0.007765,0.0077549,0.0077641,7262792,56352.342602,265,5152704,39983.2083516
2024-08-31 03:10:00,1725073800,0.0077641,0.0077649,0.0077579,0.0077608,15808259,122697.2778205,442,6191393,48054.4331655


In [4]:
from controllers.generic.dneitor import DneitorConfig
from decimal import Decimal


# Controller configuration
connector_name = "binance_perpetual"
trading_pair = "1000PEPE-USDT"
total_amount_quote = 1000
take_profit = 0.002
prices = [0.0068, 0.0075, 0.0077]
amounts_quote_pct = [1, 0.5, 0.2]
activation_bounds = 0.03
max_open_orders = 50
start = int(datetime.datetime(2024, 8, 24, 5).timestamp())
end = int(datetime.datetime(2024, 8, 31, 3).timestamp())
backtesting_resolution = "1m"


# Creating the instance of the configuration and the controller
config = DneitorConfig(
    connector_name=connector_name,
    trading_pair=trading_pair,
    total_amount_quote=Decimal(total_amount_quote),
    take_profit=Decimal(take_profit),
    prices=prices,
    amounts_quote_pct=amounts_quote_pct,
    activation_bounds=activation_bounds,
    max_open_orders=max_open_orders)

In [5]:
from research_notebooks.dneitor.dneitor_bt import DneitorBacktesting

backtester = DneitorBacktesting()
backtester.backtesting_data_provider.start_time = start
backtester.backtesting_data_provider.end_time = end

# Running the backtesting this will output a backtesting result object that has built in methods to visualize the results
backtesting_result = await backtesting.run_backtesting(
    config=config,
    start=start,
    end=end,
    backtesting_resolution=backtesting_resolution,
    backtester=backtester
)

AttributeError: 'BacktestingDataProvider' object has no attribute 'get_trading_rules'

In [ ]:
# Let's see what is inside the backtesting results
print(backtesting_result.get_results_summary())
backtesting_result.get_backtesting_figure()

In [ ]:
# 2. The executors dataframe: this is the dataframe that contains the information of the orders that were executed
import pandas as pd

executors_df = backtesting_result.executors_df
executors_df.head()

### Backtesting Analysis

### Scatter of PNL per Trade
This bar chart illustrates the PNL for each individual trade. Positive PNLs are shown in green and negative PNLs in red, providing a clear view of profitable vs. unprofitable trades.


In [ ]:
import plotly.express as px

# Create a new column for profitability
executors_df['profitable'] = executors_df['net_pnl_quote'] > 0

# Create the scatter plot
fig = px.scatter(
    executors_df,
    x="timestamp",
    y='net_pnl_quote',
    title='PNL per Trade',
    color='profitable',
    color_discrete_map={True: 'green', False: 'red'},
    labels={'timestamp': 'Timestamp', 'net_pnl_quote': 'Net PNL (Quote)'},
    hover_data=['filled_amount_quote', 'side']
)

# Customize the layout
fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Net PNL (Quote)",
    legend_title="Profitable",
    font=dict(size=12, color="white"),
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0.8)',  # Dark background
    paper_bgcolor='rgba(0,0,0,0.8)',  # Dark background for the entire plot area
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray")
)

# Add a horizontal line at y=0 to clearly separate profits and losses
fig.add_hline(y=0, line_dash="dash", line_color="lightgray")

# Show the plot
fig.show()

### Histogram of PNL Distribution
The histogram displays the distribution of PNL values across all trades. It helps in understanding the frequency and range of profit and loss outcomes.


In [ ]:
fig = px.histogram(executors_df, x='net_pnl_quote', title='PNL Distribution')
fig.show()


# Conclusion
We can see that the indicator has potential to bring good signals to trade and might be interesting to see how we can design a market maker that shifts the mid price based on this indicator.
A lot of the short signals are wrong but if we zoom in into the loss signals we can see that the losses are not that big and the wins are bigger and if we had implemented the trailing stop feature probably a lot of them are going to be profits.

# Next steps
- Filter only the loss signals and understand what you can do to prevent them
- Try different configuration values for the indicator
- Test in multiple markets, pick mature markets like BTC-USDT or ETH-USDT and also volatile markets like DOGE-USDT or SHIB-USDT